In [44]:
# 可视化 对比度 饱和度等特征
import cv2
import numpy as np
import matplotlib.pyplot as plt
import sklearn

In [24]:

def standardization(data):
    mu = np.mean(data, axis=0)
    sigma = np.std(data, axis=0)
    return (data - mu) / sigma

In [25]:
# 加载训练集
train_root_path = "G:\\vscode_workspace\\Weather_Recognition\\data_split_v6\\train\\"
train_x = []
train_y = []
label_dict = {'Cloud': 0, 'Sunny': 1}
if not os.path.exists(train_root_path):
    print(train_root_path)
    raise FileNotFoundError
with os.scandir(train_root_path) as root:
    for emotion in root:
        if emotion.name not in label_dict:
            continue
        label = label_dict[emotion.name]
        with os.scandir(emotion.path) as fold:
            for img in fold:
                img = cv2.imread(img.path)
                img = cv2.resize(img, (480, 480))
                img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
                img_lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
                hist_s = cv2.calcHist([img_hsv], [1], None, [256], [0.0, 255.0])
                standardization(hist_s)
                hist_l = cv2.calcHist([img_lab], [0], None, [256], [0.0, 255.0])
                standardization(hist_l)
                train_x.append(np.concatenate((hist_s, hist_l), axis=0))
                train_y.append(label)

In [34]:
# 加载测试集
test_root_path = "G:\\vscode_workspace\\Weather_Recognition\\data_split_v6\\test\\"
test_x = []
test_y = []
label_dict = {'Cloud': 0, 'Sunny': 1}
if not os.path.exists(test_root_path):
    print(test_root_path)
    raise FileNotFoundError
with os.scandir(test_root_path) as root:
    for emotion in root:
        if emotion.name not in label_dict:
            continue
        label = label_dict[emotion.name]
        with os.scandir(emotion.path) as fold:
            for img in fold:
                img = cv2.imread(img.path)
                img = cv2.resize(img, (480, 480))
                img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
                img_lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
                hist_s = cv2.calcHist([img_hsv], [1], None, [256], [0.0, 255.0])
                hist_l = cv2.calcHist([img_lab], [0], None, [256], [0.0, 255.0])
                standardization(hist_s)
                standardization(hist_l)
                test_x.append(np.concatenate((hist_s, hist_l), axis=0))
                test_y.append(label)
test_x = np.array(test_x)
test_y = np.array(test_y)
test_x = test_x.squeeze()

In [27]:
print(len(train_x), len(train_y))
train_x = np.array(train_x)
train_y = np.array(train_y)
print(train_x.shape)
print(train_y.shape)

15570 15570
(15570, 512, 1)
(15570,)


In [28]:
train_x = train_x.squeeze()
print(train_x.shape)

(15570, 512)


In [17]:
print(test_x.shape, test_y.shape)

(4470, 512) (4470,)


In [48]:
from sklearn.ensemble import AdaBoostClassifier

AB = AdaBoostClassifier(n_estimators=300)
AB.fit(train_x,train_y)

AdaBoostClassifier(n_estimators=300)

In [49]:
# 保存模型
import pickle
save_path = "adaboost_s_l_norm_densenet121_300.pkl"
with open(save_path, 'wb') as pfile:
    pickle.dump(AB, pfile)

In [54]:
from sklearn.metrics import accuracy_score
# train_acc_svc = accuracy_score(AB.predict(train_x), train_y)
# test_acc_svc = accuracy_score(AB.predict(test_x), test_y)
val_acc_svc = accuracy_score(AB.predict(val_x), val_y)
# print(train_acc_svc)
# print(test_acc_svc)
print(val_acc_svc)

0.5681818181818182


In [42]:
# 加载公司测试集
val_root_path = "G:\\vscode_workspace\\Weather_Recognition\\val_imgs_2\\"
val_x = []
val_y = []
label_dict = {'Cloud': 0, 'Sunny': 1}
if not os.path.exists(val_root_path):
    print(val_root_path)
    raise FileNotFoundError
with os.scandir(val_root_path) as root:
    for emotion in root:
        if emotion.name not in label_dict:
            continue
        label = label_dict[emotion.name]
        with os.scandir(emotion.path) as fold:
            for img in fold:
                img = cv2.imread(img.path)
                img = cv2.resize(img, (480, 480))
                img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
                img_lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
                hist_s = cv2.calcHist([img_hsv], [1], None, [256], [0.0, 255.0])
                hist_l = cv2.calcHist([img_lab], [0], None, [256], [0.0, 255.0])
                standardization(hist_s)
                standardization(hist_l)
                val_x.append(np.concatenate((hist_s, hist_l), axis=0))
                val_y.append(label)
val_x = np.array(val_x)
val_y = np.array(val_y)
val_x = val_x.squeeze()

In [46]:
import pickle
with open('data_train_sl_512_densenet121_1024.pkl', 'rb') as f:
    my_dict = pickle.load(f)

# my_dict = {'sl_features': sl_features, 'features_blobs': features_blobs, 'labels': label}
sl_features = my_dict['sl_features']
features_blobs = my_dict['features_blobs']
label = my_dict['labels']

In [47]:
train_x = np.concatenate((sl_features, features_blobs), axis=1)
train_y = label
print(train_x.shape, train_y.shape)

(15570, 1536) (15570,)


In [51]:
with open('data_test_sl_512_densenet121_1024.pkl', 'rb') as f:
    my_dict = pickle.load(f)

# my_dict = {'sl_features': sl_features, 'features_blobs': features_blobs, 'labels': label}
sl_features = my_dict['sl_features']
features_blobs = my_dict['features_blobs']
label = my_dict['labels']

test_x = np.concatenate((sl_features, features_blobs), axis=1)
test_y = label
print(test_x.shape, test_y.shape)

(4470, 1536) (4470,)


In [53]:
with open('data_val_sl_512_densenet121_1024.pkl', 'rb') as f:
    my_dict = pickle.load(f)

# my_dict = {'sl_features': sl_features, 'features_blobs': features_blobs, 'labels': label}
sl_features = my_dict['sl_features']
features_blobs = my_dict['features_blobs']
label = my_dict['labels']

val_x = np.concatenate((sl_features, features_blobs), axis=1)
val_y = label
print(val_x.shape, val_y.shape)

(440, 1536) (440,)
